In [1]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.utilities import WikipediaAPIWrapper
from duckduckgo_search import DDGS 
import logging
import time

from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import os
from typing import List, Optional

logging.basicConfig(level=logging.INFO)

# 1. 환경 설정 및 LLM 초기화
llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4-turbo"
)

# 2. 커스텀 도구 정의 및 구현

# 2.1. Wikipedia 검색 도구 (APIWrapper 기반)
wikipedia_wrapper = WikipediaAPIWrapper()
# 도구 이름의 공백을 밑줄(_)로 변경하여 OpenAI Function Calling 패턴을 충족시킵니다.
wikipedia_tool = Tool(
    name="Wikipedia_Search",
    func=wikipedia_wrapper.run,
    description="유용한 정보를 찾기 위해 위키피디아에서 검색할 때 사용합니다. 입력은 검색 쿼리여야 합니다.",
)

# 2.2. DuckDuckGo 검색 도구 (안정화된 커스텀 함수 - DDGS().text() 사용)
def duckduckgo_search_tool(query: str, max_results: int = 3) -> str:
    MAX_RETRIES = 3
    
    for attempt in range(MAX_RETRIES):
        try:
            results = []
            # DDGS 클래스를 인스턴스화합니다.
            with DDGS() as ddgs:
                for r in ddgs.text(keywords=query, region='wt-wt', timelimit='y'):
                    results.append(r)
                    # max_results에 도달하면 루프를 명시적으로 중단합니다.
                    if len(results) >= max_results:
                        break

            if results:
                # 결과가 성공적으로 검색되면 포맷팅하여 반환합니다.
                formatted_results = []
                for i, result in enumerate(results):
                    # LLM이 읽기 쉽고 정보를 추출하기 쉽게 포맷팅합니다.
                    formatted_results.append(
                        f"Result {i+1}:\n"
                        f"  Title: {result.get('title', 'N/A')}\n"
                        f"  Snippet: {result.get('body', 'N/A')}\n"
                        f"  Link: {result.get('href', 'N/A')}"
                    )
                logging.info(f"DuckDuckGo search successful on attempt {attempt + 1}.")
                return "\n---\n".join(formatted_results)
            
            # 결과가 없지만 예외가 발생하지 않은 경우 (일반적으로 실패로 간주하고 재시도)
            logging.warning(f"DuckDuckGo search returned no results for query '{query}' on attempt {attempt + 1}.")

        except Exception as e:
            logging.error(f"DuckDuckGo search exception for query '{query}' on attempt {attempt + 1}: {e}")
            
            # 마지막 시도에서 예외가 발생하면 예외 객체를 명시적으로 반환
            if attempt == MAX_RETRIES - 1:
                return f"DuckDuckGo search failed after {MAX_RETRIES} attempts due to an internal API error: {e}"

        # 재시도 로직 (마지막 시도가 아니면서, 예외가 발생했거나 결과가 없는 경우)
        if attempt < MAX_RETRIES - 1:
            # 마지막 시도가 아니면 지수 백오프 대기 후 재시도 (1초, 2초, 4초)
            delay = 2 ** attempt
            logging.info(f"Retrying DuckDuckGo search in {delay} seconds...")
            time.sleep(delay)
    
    # 모든 시도가 결과 없이 종료되었을 때 최종 반환 (모든 시도에서 예외가 발생하지 않았지만 결과도 없는 경우)
    return "DuckDuckGo search failed: No results found after multiple attempts or an unknown error occurred."

ddg_tool = Tool(
    name="DuckDuckGo_Search",
    func=duckduckgo_search_tool,
    description="최신 정보를 찾거나 웹사이트 링크를 얻기 위해 덕덕고에서 검색할 때 사용합니다. 입력은 검색 쿼리여야 하며, 상위 3개의 결과를 반환합니다.",
)

# 2.3. 웹사이트 스크래핑 도구 (커스텀 래퍼)
def scrape_website(url: str) -> str:
    """주어진 URL에서 텍스트 콘텐츠를 스크래핑하고 추출합니다."""
    try:
        # WebBaseLoader를 사용하여 URL에서 문서를 로드합니다.
        loader = WebBaseLoader(url)
        data = loader.load()

        # 추출된 문서의 page_content를 결합합니다.
        full_text = "\n\n".join([doc.page_content for doc in data])
        
        # 텍스트가 너무 길면 에이전트 추론을 위해 일부만 반환합니다.
        if len(full_text) > 4000:
            return f"Successfully scraped the website. Extracted content (first 4000 characters): {full_text[:4000]}..."
        
        return f"Successfully scraped the website. Extracted content: {full_text}"
    except Exception as e:
        return f"Error scraping the website {url}: {e}"

scrape_tool = Tool(
    name="Web_Scraper", 
    func=scrape_website,
    description="특정 웹사이트의 URL을 받아서 텍스트 콘텐츠를 스크랩하고 추출할 때 사용합니다. DuckDuckGo 검색 결과에서 얻은 URL에만 사용하세요. 입력은 단일 URL이어야 합니다.",
)

# 2.4. 리서치 결과 파일 저장 도구 (커스텀)
def save_research_to_file(research_content: str) -> str:
    """주어진 텍스트 콘텐츠를 'research_results.txt' 파일에 저장합니다."""
    filepath = "research_results.txt"
    try:
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(research_content)
        return f"Research successfully saved to file: {filepath}"
    except Exception as e:
        return f"Error saving research to file {filepath}: {e}"

save_tool = Tool(
    name="File_Saver", 
    func=save_research_to_file,
    description="모든 리서치 작업이 완료된 후 최종 결과를 .txt 파일에 저장할 때 사용합니다. 입력은 저장할 최종 요약 텍스트여야 합니다. 이 도구를 사용한 후 최종 응답을 제공해야 합니다.",
)

# 3. 에이전트 초기화

# 모든 커스텀 도구들을 리스트로 묶습니다.
research_tools = [wikipedia_tool, ddg_tool, scrape_tool, save_tool]

# AgentType.OPENAI_FUNCTIONS는 LLM의 함수 호출 기능을 활용하여
# 어떤 도구를 사용해야 하는지, 어떤 인수로 호출해야 하는지를 결정하는 데 탁월합니다.
agent = initialize_agent(
    tools=research_tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True, # 에이전트의 사고 과정(Thought/Action)을 자세히 출력합니다.
    handle_parsing_errors=True, # 파싱 오류가 발생해도 에이전트가 자체적으로 수정하도록 합니다.
)

# 4. 에이전트 실행

# 파일 저장을 유도하는 지침을 research_query에 명시적으로 추가합니다.
research_query = (
    "Research about the XZ backdoor. "
    "After gathering and summarizing the information, "
    "you MUST use the 'File_Saver' tool to save the final summary before providing the Final Answer."
)

print(f"\n--- Starting Research for: {research_query} ---\n")

# 에이전트 실행
try:
    # agent.run은 최종 응답을 문자열로 반환합니다.
    final_output = agent.run(research_query)
    print("\n--- Research Agent Finished ---")
    print(f"\nFinal Agent Output (after saving file): {final_output}")
except Exception as e:
    # 예상치 못한 오류 발생 시 디버깅을 돕기 위해 예외를 출력합니다.
    print(f"\n--- An unexpected error occurred: {e} ---")

# 저장된 파일 내용을 확인하는 코드 (디버깅 용도)
try:
    with open("research_results.txt", "r", encoding="utf-8") as f:
        print("\n--- Content of research_results.txt: ---")
        print(f.read())
except FileNotFoundError:
    print("\n--- File 'research_results.txt' was not created. ---")


--- Starting Research for: Research about the XZ backdoor. After gathering and summarizing the information, you MUST use the 'File_Saver' tool to save the final summary before providing the Final Answer. ---



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGo_Search` with `XZ backdoor`




INFO:httpx:HTTP Request: POST https://duckduckgo.com "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://links.duckduckgo.com/d.js?q=XZ%20backdoor&kl=wt-wt&l=wt-wt&s=0&df=y&vqd=4-330282670165161608592410124150009399120&o=json&sp=0&ex=-1 "HTTP/2 202 Accepted"
INFO:httpx:HTTP Request: GET https://links.duckduckgo.com/d.js?q=XZ%20backdoor&kl=wt-wt&l=wt-wt&s=0&df=y&vqd=4-330282670165161608592410124150009399120&o=json&sp=0&ex=-1 "HTTP/2 202 Accepted"
INFO:httpx:HTTP Request: GET https://links.duckduckgo.com/d.js?q=XZ%20backdoor&kl=wt-wt&l=wt-wt&s=0&df=y&vqd=4-330282670165161608592410124150009399120&o=json&sp=0&ex=-1 "HTTP/2 202 Accepted"
ERROR:root:DuckDuckGo search exception for query 'XZ backdoor' on attempt 1: 
INFO:root:Retrying DuckDuckGo search in 1 seconds...
INFO:httpx:HTTP Request: POST https://duckduckgo.com "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://links.duckduckgo.com/d.js?q=XZ%20backdoor&kl=wt-wt&l=wt-wt&s=0&df=y&vqd=4-330282670165161608592410124150009399120&o=json&

DuckDuckGo search failed after 3 attempts due to an internal API error: 
Invoking: `Wikipedia_Search` with `XZ backdoor`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
S